In [2]:
# Import dependencies
import qibo
from qibo.models import Circuit
from qibo import gates
import numpy as np
from copy import deepcopy

# Generate GHZ-like state with relative phase

This circuit aims to generate the state $|\psi\rang=1/\sqrt 2(|000\rang + i|111\rang)$

In [4]:
GHZ_3 = Circuit(3)

# Apply gates according to the given circuit diagram
GHZ_3.add([
    gates.H(1),
    gates.CNOT(1, 2),
    gates.H(1), 

    gates.H(0),
    gates.CNOT(0,2),
    gates.H(0),
    gates.H(2),
    gates.S(2),

    gates.M(0,1,2)

])

print(GHZ_3.draw())

q0: ─────H─o─H───M─
q1: ─H─o─H─|─────M─
q2: ───X───X─H─S─M─


In [7]:
# Z measurement results on the generation circuit
GHZ_result = GHZ_3(nshots=1000)
GHZ_result.frequencies()

Counter({'111': 507, '000': 493})

# Measurement
For 3 qubits, the Mermin polynomial reads
$$
\begin{align}
M_3 = (a_1a_2a_3' + a_1a_2'a_3 + a_1'a_2a_3 - a_1'a_2'a_3')
\end{align}
$$

, where $a_i$ represents $\sigma_x$ on the ith qubit, while $a_i'$ represents $\sigma_y$ on the same qubit.

By virtue of symmetry, we can simplify the measurement to:

$$
\begin{align}
M_3 &= 3(a_1a_2a_3') - a_1'a_2'a_3'
\end{align}
$$

which only requires 2 experiments.


The classical bound of $\lang M_3\rang^{LR}\le 2$ and the quantum bound $\lang M_3\rang ^{QM} \le 4$

In [8]:
def GHZ_3_generate():
    circuit = Circuit(3)
    circuit.add([
    gates.H(1),
    gates.CNOT(1, 2),
    gates.H(1), 

    gates.H(0),
    gates.CNOT(0,2),
    gates.H(0),
    gates.H(2),
    gates.S(2),
])
    return circuit

In [9]:
nshots = 10000
meas_xxy = GHZ_3_generate()
meas_xxy.add(gates.SDG(2))
meas_xxy.add([gates.H(i) for i in range(3)])
meas_xxy.add(gates.M(0,1,2))
print(meas_xxy.draw())
xxy_result = meas_xxy(nshots=nshots)
xxy_result.frequencies()


q0: ─────H─o─H─H───────M─
q1: ─H─o─H─|─H─────────M─
q2: ───X───X─H─S─SDG─H─M─


Counter({'011': 2639, '101': 2472, '000': 2467, '110': 2422})

In [10]:
meas_yyy = GHZ_3_generate()
meas_yyy.add([gates.SDG(i) for i in range(3)])
meas_yyy.add([gates.H(i) for i in range(3)])
meas_yyy.add(gates.M(0,1,2))
print(meas_yyy.draw())
yyy_result = meas_yyy(nshots=nshots)
yyy_result.frequencies()


q0: ─────H─o─H───SDG─H─────M─
q1: ─H─o─H─|─SDG─H─────────M─
q2: ───X───X─H───S───SDG─H─M─


Counter({'100': 2607, '111': 2505, '010': 2462, '001': 2426})

# Results analysis

In [11]:
import pandas as pd

In [12]:
def count_to_frequency(counter, total_shots: int = None):
    if total_shots is None:
        total_shots = sum(counter.values())
    frequencies = {outcome: count / total_shots for outcome, count in counter.items()}
    return frequencies

In [13]:
xxy_frequency = count_to_frequency(xxy_result.frequencies())
yyy_frequency = count_to_frequency(yyy_result.frequencies())
data = {
    'XXXXX' : dict(xxy_frequency),
    'XXXYY' : dict(yyy_frequency),
}
df = pd.DataFrame(data)
df = df.fillna(0)
print(df)

      XXXXX   XXXYY
000  0.2467  0.0000
011  0.2639  0.0000
101  0.2472  0.0000
110  0.2422  0.0000
001  0.0000  0.2426
010  0.0000  0.2462
100  0.0000  0.2607
111  0.0000  0.2505


To translate the frequencies into the expected values, we group the resultant states by parity

In [14]:
def calculate_parity(outcome):
    """Determine the parity of string outcome. 1 if odd, 0 iff even."""
    return outcome.count('1')%2

def calculate_expectancy(frequency_dict):
    """Calculate the expected value based on parity"""
    expect = 0
    for outcome, probability in frequency_dict.items():
        if calculate_parity(outcome) == 0: # Even parity
            expect += probability
        else:
            expect -= probability
    return expect

In [15]:
A = calculate_expectancy(xxy_frequency)
B = calculate_expectancy(yyy_frequency)
print(A,B)
M3 = 3*A - B
print(M3)

1.0 -1.0
4.0
